In [25]:
import numpy as np
import pandas as pd
import pcl

def load_pcd_to_numpy(pcd_path):
    cloud = pcl.load(pcd_path)
    return np.asarray(cloud)

def compute_z_variance(pcd_data, min_x, max_x, min_y, max_y):
    filtered_points = pcd_data[(pcd_data[:, 0] >= min_x) & (pcd_data[:, 0] <= max_x) &
                               (pcd_data[:, 1] >= min_y) & (pcd_data[:, 1] <= max_y)]
    z_values = filtered_points[:, 2]
    variance = np.var(z_values)
    return variance

def main():
    # Load PCD data
    pcd_path = "/home/rtlink/jiwon/socap_ws/src/final/BumpiPatch/result/point_cloud_sample_1/cropped3_final.pcd"
    pcd_data = load_pcd_to_numpy(pcd_path)
    
    # Load CSV data
    csv_path = "/home/rtlink/jiwon/socap_ws/src/final/BumpiPatch/result/point_cloud_sample_1/cropped3_final.csv"
    df = pd.read_csv(csv_path)

    # Compute variance for each patch
    result = {}
    for index, row in df.iterrows():
        variance = compute_z_variance(pcd_data, row['min_x'], row['max_x'], row['min_y'], row['max_y'])
        class_name = row['class']
        
        if class_name not in result:
            result[class_name] = []
        result[class_name].append(variance)
        
    # Print results
    for class_name, variances in result.items():
        print(f"Class: {class_name}")
        print(f"Z Variance: {np.mean(variances)} ± {np.std(variances)}")
        
if __name__ == "__main__":
    main()


Class: 4
Z Variance: 36.136871337890625 ± 38.48357391357422
Class: 3
Z Variance: 0.09521334618330002 ± 0.06317336857318878
Class: 2
Z Variance: 0.032967034727334976 ± 0.0721006914973259
Class: 1
Z Variance: 0.003929831553250551 ± 0.003880613949149847


In [27]:
def main():
    # Load PCD data
    pcd_path = "/home/rtlink/jiwon/socap_ws/src/final/BumpiPatch/result/point_cloud_sample_3/gravel_quarry_final.pcd"
    pcd_data = load_pcd_to_numpy(pcd_path)
    
    # Load CSV data
    csv_path = "/home/rtlink/jiwon/socap_ws/src/final/BumpiPatch/result/point_cloud_sample_3/gravel_quarry_final.csv"
    df = pd.read_csv(csv_path)

    # Compute variance for each patch
    result = {}
    for index, row in df.iterrows():
        variance = compute_z_variance(pcd_data, row['min_x'], row['max_x'], row['min_y'], row['max_y'])
        class_name = row['class']
        
        if class_name not in result:
            result[class_name] = []
        result[class_name].append(variance)
        
    # Print results
    for class_name, variances in result.items():
        print(f"Class: {class_name}")
        print(f"Z Variance: {np.mean(variances)} ± {np.std(variances)}")
        
if __name__ == "__main__":
    main()


Class: 4
Z Variance: 0.695554792881012 ± 2.409458875656128
Class: 2
Z Variance: 0.06936217844486237 ± 0.06885690242052078
Class: 3
Z Variance: 0.15170742571353912 ± 0.07738134264945984
Class: 1
Z Variance: 0.009609036147594452 ± 0.021006347611546516


In [2]:
import numpy as np
import pandas as pd
import open3d as o3d
from sklearn.linear_model import RANSACRegressor

def load_pcd_to_numpy(pcd_path):
    cloud = o3d.io.read_point_cloud(pcd_path)
    return np.asarray(cloud.points)

def compute_residual(pcd_data, min_x, max_x, min_y, max_y):
    # Filter points within the patch
    filtered_points = pcd_data[(pcd_data[:, 0] >= min_x) & (pcd_data[:, 0] <= max_x) &
                               (pcd_data[:, 1] >= min_y) & (pcd_data[:, 1] <= max_y)]
    
    # Check if there are enough points for RANSAC
    if len(filtered_points) < 10:  # or another threshold you consider appropriate
        return np.nan

    xy_values = filtered_points[:, :2]
    z_values = filtered_points[:, 2]
    
    # RANSAC to find representative plane
    ransac = RANSACRegressor()
    ransac.fit(xy_values, z_values)
    predicted_z = ransac.predict(xy_values)
    
    # Compute residuals
    residuals = np.abs(predicted_z - z_values)
    
    return np.mean(residuals)


def main():
    # Load PCD data
    pcd_path = "/home/rtlink/jiwon/socap_ws/src/final/BumpiPatch/result/point_cloud_sample_1/cropped3_final.pcd"
    pcd_data = load_pcd_to_numpy(pcd_path)
    
    # Load CSV data
    csv_path = "/home/rtlink/jiwon/socap_ws/src/final/BumpiPatch/result/point_cloud_sample_1/cropped3_final.csv"
    df = pd.read_csv(csv_path)

    # Compute residual for each patch
    result = {}
    for index, row in df.iterrows():
        residual = compute_residual(pcd_data, row['min_x'], row['max_x'], row['min_y'], row['max_y'])
        class_name = row['class']
        
        if class_name not in result:
            result[class_name] = []
        result[class_name].append(residual)
        
    # Print results
    for class_name, residuals in result.items():
        print(f"Class: {class_name}")
        print(f"Residual: {np.nanmean(residuals)} ± {np.nanstd(residuals)}")  # using nanmean to handle potential NaNs
        
if __name__ == "__main__":
    main()


Class: 4
Residual: 3.6157360220134676 ± 3.1031753329853755
Class: 3
Residual: 0.1718975661855622 ± 0.06237137181468443
Class: 2
Residual: 0.07881418083746701 ± 0.07820823198161649
Class: 1
Residual: 0.036929414558946126 ± 0.01517940791382956


In [3]:
def main():
    # Load PCD data
    pcd_path = "/home/rtlink/jiwon/socap_ws/src/final/BumpiPatch/result/point_cloud_sample_3/gravel_quarry_final.pcd"
    pcd_data = load_pcd_to_numpy(pcd_path)
    
    # Load CSV data
    csv_path = "/home/rtlink/jiwon/socap_ws/src/final/BumpiPatch/result/point_cloud_sample_3/gravel_quarry_final.csv"
    df = pd.read_csv(csv_path)

    # Compute residual for each patch
    result = {}
    for index, row in df.iterrows():
        residual = compute_residual(pcd_data, row['min_x'], row['max_x'], row['min_y'], row['max_y'])
        class_name = row['class']
        
        if class_name not in result:
            result[class_name] = []
        result[class_name].append(residual)
        
    # Print results
    for class_name, residuals in result.items():
        print(f"Class: {class_name}")
        print(f"Residual: {np.nanmean(residuals)} ± {np.nanstd(residuals)}")  # using nanmean to handle potential NaNs
        
if __name__ == "__main__":
    main()

Class: 4
Residual: 0.17596661405673977 ± 0.3176224566710943
Class: 2
Residual: 0.09621773494475044 ± 0.06164230962966713
Class: 3
Residual: 0.09324939902251732 ± 0.059822804064348506
Class: 1
Residual: 0.03603795728456698 ± 0.01694290686705635


In [6]:
import pandas as pd

def compute_class_percentage(csv_path):
    # Load CSV data
    df = pd.read_csv(csv_path)
    
    # Calculate class percentages
    class_counts = df['class'].value_counts()
    class_percentages = class_counts / len(df) * 100
    
    return class_percentages

def main():
    csv_path = "/home/rtlink/jiwon/socap_ws/src/final/BumpiPatch/result/point_cloud_sample_1/cropped3_final.csv"
    
    percentages = compute_class_percentage(csv_path)
    for class_name, percentage in percentages.items():
        print(f"Class: {class_name}, Percentage: {percentage:.2f}%")
        
if __name__ == "__main__":
    main()


Class: 4, Percentage: 68.21%
Class: 1, Percentage: 18.24%
Class: 2, Percentage: 12.78%
Class: 3, Percentage: 0.77%
